In [ ]:
%matplotlib inline

# system
import os
import importlib
import datetime
import sys

# scipy
import numpy as np # type: ignore
from scipy import signal # type: ignore

# matplotlib
import matplotlib.pyplot as plt # type: ignore

# grandlib
import grand.dataio.root_trees as rt # type: ignore

#import tools
import database.tools as tools

plt.style.use('/pbs/home/p/pcorrea/tools/matplotlib_style_sans-serif.txt')

In [ ]:
du  = 1021
th1 = 55
th2 = 35
sfe = 100 # samples from trace edge

title = f'DU: {du}, TH1: {th1}, TH2: {th2}'


file = f'/sps/grand/pcorrea/nutrig/database/bkg/pretrigger_thresh_test/trigger_count_du_{du}_th1_{th1}_th2_{th2}_sfe_{sfe}.npz'

f = np.load(file)

n_entries_trig_file = f['n_entries_trig']
n_entries_tot_file  = f['n_entries_tot']
du_seconds          = f['du_seconds']
files               = f['files']

n_trig = n_entries_trig_file.sum()
n_tot  = n_entries_tot_file.sum()

In [ ]:
rate_max       = 1e3 # [Hz]
time_per_trace = (1024-2*sfe)*2e-9 # [s]
time_tot_trace = n_tot*time_per_trace
n_trig_max     = int( rate_max*time_tot_trace )

n_trig_max, n_trig, n_tot

In [ ]:
seconds_in_day         = 24*3600 # [s]
time_recorded_in_day   = time_per_trace * seconds_in_day/10 # 10 second data
fraction_time_recorded = time_recorded_in_day/seconds_in_day

tz    = datetime.timezone(datetime.timedelta(hours=8)) # UTC+8
dates = [datetime.datetime.fromtimestamp(du_second,tz=tz) for du_second in du_seconds]

bin_edge_first = int( datetime.datetime(dates[0].year,dates[0].month,dates[0].day,tzinfo=tz).timestamp() ) # [s]
bin_edge_last  = int( datetime.datetime(dates[-1].year,dates[-1].month,dates[-1].day,tzinfo=tz).timestamp() ) # [s]

bin_width      = seconds_in_day
bin_edges      = np.arange(bin_edge_first,bin_edge_last+seconds_in_day+1,seconds_in_day)
bin_centers    = (bin_edges[1:] + bin_edges[:-1])/2.

hist, bin_edges = np.histogram(du_seconds,bins=bin_edges)

In [ ]:
fig, ax = plt.subplots()

ax.bar(bin_centers,np.cumsum(hist),width=bin_width,align='center',color='b',label='GP13 data')
ax.axhline(n_trig_max,ls='--',color='k',label='Limit for average 1 kHz rate')

xticks      = ax.get_xticks()
xticklables = [datetime.datetime.fromtimestamp(xtick,tz=tz).date() for xtick in xticks]

ax.set_xticklabels(xticklables)
ax.set_ylabel('Cumulative number of triggers')

ax.set_title(title)

ax.legend(loc='center left')

plt.xticks(rotation=45)
plt.show()


fig, ax = plt.subplots()

ax.bar(bin_centers,hist/seconds_in_day/fraction_time_recorded,width=bin_width,align='center',color='b',label='GP13 data')
ax.axhline(1000,ls='--',color='k',label='Hardware limit')
ax.axhline(np.mean(hist/seconds_in_day/fraction_time_recorded),ls='--',color='r',label='GP13 data average')

xticks      = ax.get_xticks()
xticklables = [datetime.datetime.fromtimestamp(xtick,tz=tz).date() for xtick in xticks]

ax.set_xticklabels(xticklables)
ax.set_ylabel('Average pulse rate [Hz]')

ax.set_title(title)

ax.legend()

plt.xticks(rotation=45)
plt.show()

In [ ]:
bin_edges       = np.arange(0,24,1)
bin_centers     = (bin_edges[1:] + bin_edges[:-1])/2.

hist, bin_edges = np.histogram([date.hour for date in dates],bins=bin_edges)


fig, ax = plt.subplots()

ax.bar(bin_centers,hist,width=1,align='center',color='b',label='GP13 data')

ax.set_xlabel('Local time (UTC+8)')
ax.set_ylabel('Counts')

ax.set_title(title)

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.bar(bin_centers,np.cumsum(hist),width=bin_width,align='center',color='b',label='GP13 data')
ax.axhline(n_trig_max,ls='--',color='k',label='Limit for average 1 kHz rate')

xticks      = ax.get_xticks()
xticklables = [datetime.datetime.fromtimestamp(xtick,tz=tz).date() for xtick in xticks]

ax.set_xticklabels(xticklables)
ax.set_ylabel('Cumulative number of triggers')

ax.set_title(title)

ax.legend(loc='center left')

plt.xticks(rotation=45)
plt.show()

In [ ]:
traces = np.zeros( (n_trig,3,1024) )
mask   = np.arange(0,1300,13)

for i in mask: #n_trig
    print(i)
    with np.load(files[i]) as f:
        mask_trig = np.argwhere( np.logical_and( f['du_ids']==du,f['du_seconds']==du_seconds[i] ) ).flatten()
        traces[i] = f['traces'][mask_trig]

In [ ]:
importlib.reload(tools)

traces_filtered = tools.filter_traces(traces,freq_highpass=50,freqs_notch=[50.2,55.1,126],bw_notch=[1.,1.,25.]) #freqs_notch=[50.2,55.1,118.8,124.4,134],bw_notch=[1.,1.,2.,3.,10.]
fft_filtered    = np.fft.rfft(traces_filtered)
freq            = np.fft.rfftfreq(1024,d=1/500)

In [ ]:
for trace,file in zip(traces_filtered[mask[:]],files[mask]):

    windows_x, trigger_times_x = tools.search_windows(trace[0],threshold1=th1,threshold2=th2,samples_from_trace_edge=sfe)
    windows_y, trigger_times_y = tools.search_windows(trace[1],threshold1=th1,threshold2=th2,samples_from_trace_edge=sfe)

    fig, ax = plt.subplots(3,1,sharex=True)

    ax[0].plot(trace[0],color='b',label='X')
    ax[1].plot(trace[1],color='m',label='Y')
    ax[2].plot(trace[2],color='r',label='Z')

    if len(trigger_times_x):
        ax[0].axvline(trigger_times_x[0],color='k',ls='--')
    if len(trigger_times_y):
        ax[1].axvline(trigger_times_y[0],color='k',ls='--')

    ax[2].set_xlabel('ADC samples')
    ax[1].set_ylabel('ADC counts')

    ax[0].legend(frameon=True,framealpha=.8)
    ax[1].legend(frameon=True,framealpha=.8)
    ax[2].legend(frameon=True,framealpha=.8)

    ax[0].set_title(os.path.basename(file),fontsize=14)

    #plt.xlim([350,500])

    plt.show()

In [ ]:
for trace in traces[50:60]:
    fig, ax = plt.subplots(3,1,sharex=True)

    ax[0].plot(trace[0],color='b',label='X')
    ax[1].plot(trace[1],color='m',label='Y')
    ax[2].plot(trace[2],color='r',label='Z')

    # ax[0].axhline(thresh1,color='k',alpha=.7)
    # ax[0].axhline(thresh2,color='k',alpha=.4)
    # ax[1].axhline(thresh1,color='k',alpha=.7)
    # ax[1].axhline(thresh2,color='k',alpha=.4)

    # ax[0].axvline(452,color='g',ls=':',alpha=1)
    # ax[0].axvline(452+25,color='orange',ls=':',alpha=1)

    ax[2].set_xlabel('ADC samples')
    ax[1].set_ylabel('ADC counts')

    ax[0].legend(frameon=True,framealpha=.8)
    ax[1].legend(frameon=True,framealpha=.8)
    ax[2].legend(frameon=True,framealpha=.8)

    #plt.xlim([350,500])

    plt.show()

In [ ]:
for fft in np.abs(fft_filtered[mask]):
    plt.plot(freq,fft[0],color='b',label='X')
    plt.plot(freq,fft[1],color='m',label='Y')
    #plt.plot(freq,fft[2],color='r',label='Z')
    plt.title(freq[np.argmax(fft[0])])
    plt.yscale('log')
    plt.show()

In [ ]:
np.mean([114,138])

In [ ]:
importlib.reload(tools)

traces_filtered = tools.filter_traces(traces,freq_highpass=40,freqs_notch=[50.2,55.1,126],bw_notch=[1.,1.,25.]) #134.2 122.5 freqs_notch=[119.1,125.5,131,137.2],bw_notch=[3.,1.,4.,1.]
fft_filtered    = np.fft.rfft(traces_filtered)
freq            = np.fft.rfftfreq(1024,d=1/500) #[115,139]

In [ ]:
fft = np.fft.rfft(traces)

mean_fft          = np.mean( np.abs(fft[mask]),axis=0 )
mean_fft_filtered = np.mean( np.abs(fft_filtered[mask]),axis=0 )

fig, ax = plt.subplots()

ax.plot(freq,mean_fft[0],color='b',alpha=.2)
ax.plot(freq,mean_fft[1],color='m',alpha=.2)
ax.plot(freq,mean_fft[2],color='r',alpha=.2)

ax.plot(freq,mean_fft_filtered[0],color='b',label='X',alpha=.5)
ax.plot(freq,mean_fft_filtered[1],color='m',label='Y',alpha=.5)
ax.plot(freq,mean_fft_filtered[2],color='r',label='Z',alpha=.5)

ax.set_yscale('log')
ax.set_xlabel('Frequency [MHz]')
ax.set_ylabel('Mean FFT of pulses [a.u.]')

ax.set_title(title)

ax.legend()

plt.show()

In [ ]:
idx_peak = signal.find_peaks(mean_fft_filtered[0],height=2e3)[0]
freq[idx_peak], freq[idx_peak+1]


In [ ]:
idx_peak = signal.find_peaks(mean_fft[0],height=1e3)[0]
freq[idx_peak]
